# NGBoost

In [28]:
import os
root_dir = '../../' if os.getcwd().split('/')[-1] != 'credit-now' else './'
os.chdir(root_dir)

import warnings
warnings.filterwarnings("ignore")  

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import credit_data

## 데이터 전처리
모든 열을 int형으로 변경

In [ ]:
train_old = pd.read_csv('credit_data/train_old.csv')
train_new = pd.read_csv('credit_data/train.csv')
train_old.shape, train_new.shape

((22724, 19), (26457, 26))

In [ ]:
for df in [train_old,train_new]:
    for column in df.columns.tolist():
        df[column] = df[column].astype('int64')

In [ ]:
old_data = train_old.drop(['index', 'credit'], axis=1).copy()
old_label = np.array(train_old[['credit']])

old_train, old_test, old_train_label, old_test_label = \
    model_selection.train_test_split(old_data, old_label, test_size=0.3,
                                        random_state=0, stratify=old_label) 

new_data = train_new.drop(['index', 'credit'], axis=1).copy()
new_label = np.array(train_new[['credit']])

new_train, new_test, new_train_label, new_test_label = \
    model_selection.train_test_split(new_data, new_label, test_size=0.3,
                                        random_state=0, stratify=new_label) 

## NGBoost Classifier

In [51]:
#!pip install --upgrade git+https://github.com/stanfordmlgroup/ngboost.git

In [52]:
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli

ngb_cat = NGBClassifier(Dist=k_categorical(3), verbose=False) # tell ngboost that there are 3 possible outcomes

Compare old & new

In [53]:
ngb_cat.fit(old_train, old_train_label) # Y should have only 3 values: {0,1,2}
ngb_pred_test = ngb_cat.predict(old_test)
accuracy_score(ngb_pred_test, old_test_label)

0.6406570841889117

In [54]:
ngb_cat.fit(new_train, new_train_label) # Y should have only 3 values: {0,1,2}
ngb_pred_test = ngb_cat.predict(new_test)
accuracy_score(ngb_pred_test, new_test_label)

0.6924918115394306

In [55]:
# print(round(accuracy_score(ngb_cat.predict(train_data), train_label), 3))

In [56]:
# ngb_cat.predict(test_data)

In [57]:
# ngb_cat.predict_proba(test_data)

In [62]:
ngb_cat.pred_dist(new_test).params

{'p0': array([0.14151951, 0.13084966, 0.1195992 , ..., 0.29091686, 0.11412647,
        0.10206274]),
 'p1': array([0.25072773, 0.14621426, 0.16669859, ..., 0.65859704, 0.22018458,
        0.15644116]),
 'p2': array([0.60775276, 0.72293607, 0.71370221, ..., 0.05048611, 0.66568895,
        0.7414961 ])}

In [ ]:
ngb_cat.pred_dist(new_test).params

In [ ]:
from ngboost.scores import LogScore, CRPScore 

NGBClassifier(Dist = k_categorical(3), Score = LogScore, verbose=False).fit(train_data, train_label)

NGBClassifier(random_state=RandomState(MT19937) at 0x103E7FB40, verbose=False)